In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load dataset
df = pd.read_csv("/content/heart.csv")

# Split features and target
X = df.drop("HeartDisease", axis=1)
y = df["HeartDisease"]

# Identify categorical columns
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
binary_cols = ['Sex', 'ExerciseAngina']
multi_cat_cols = list(set(categorical_cols) - set(binary_cols))

# Label encode binary columns
label_encoders = {col: LabelEncoder() for col in binary_cols}
for col in binary_cols:
    X[col] = label_encoders[col].fit_transform(X[col])

# One-hot encode remaining categorical columns
X = pd.get_dummies(X, columns=multi_cat_cols)

# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Initialize models
models = {
    "SVM": SVC(),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier()
}

# Train and evaluate before PCA
print("Accuracy before PCA:")
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"{name}: {accuracy_score(y_test, y_pred):.4f}")

# Apply PCA to retain 95% variance
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)
X_train_pca, X_test_pca = train_test_split(X_pca, test_size=0.2, random_state=42)

# Train and evaluate after PCA
print("\nAccuracy after PCA:")
for name, model in models.items():
    model.fit(X_train_pca, y_train)
    y_pred_pca = model.predict(X_test_pca)
    print(f"{name}: {accuracy_score(y_test, y_pred_pca):.4f}")


Accuracy before PCA:
SVM: 0.8804
Logistic Regression: 0.8533
Random Forest: 0.8913

Accuracy after PCA:
SVM: 0.8804
Logistic Regression: 0.8533
Random Forest: 0.8587
